In [5]:
import pandas as pd
import numpy as np
import os

from kaggle.api.kaggle_api_extended import KaggleApi
import time

In [6]:
version = 'v004' # UPDATE

In [7]:
competition = 'dm-ey-f-2024-segunda'
files_path = f'C:/Eugenio/Maestria/DMEyF/exp/competencia_3/{version}/'

In [8]:
semillas = [122219, 109279, 400391, 401537, 999961]

puntos_corte = np.linspace(10000, 11000, 5).astype(int) # Usar el mismo que en el script anterior

In [9]:
# diccionario con la semilla y el corte de cada version
combinations = {}

counter = 1
for semilla in semillas:
    for corte in puntos_corte:
        combinations[counter] = {'seed':semilla,'corte':int(corte)}
        counter += 1

# path de cada version        
files = [f'{files_path}results_{version}_{i}.csv' for i in range(1,len(combinations)+1)]

# descripcion de cada edicion
descriptions= [
    f"seed = {combinations[i]['seed']}, corte = {combinations[i]['corte']}"
    for i in range(1, len(combinations)+1)
]

In [10]:
descriptions

['seed = 122219, corte = 10000',
 'seed = 122219, corte = 10250',
 'seed = 122219, corte = 10500',
 'seed = 122219, corte = 10750',
 'seed = 122219, corte = 11000',
 'seed = 109279, corte = 10000',
 'seed = 109279, corte = 10250',
 'seed = 109279, corte = 10500',
 'seed = 109279, corte = 10750',
 'seed = 109279, corte = 11000',
 'seed = 400391, corte = 10000',
 'seed = 400391, corte = 10250',
 'seed = 400391, corte = 10500',
 'seed = 400391, corte = 10750',
 'seed = 400391, corte = 11000',
 'seed = 401537, corte = 10000',
 'seed = 401537, corte = 10250',
 'seed = 401537, corte = 10500',
 'seed = 401537, corte = 10750',
 'seed = 401537, corte = 11000',
 'seed = 999961, corte = 10000',
 'seed = 999961, corte = 10250',
 'seed = 999961, corte = 10500',
 'seed = 999961, corte = 10750',
 'seed = 999961, corte = 11000']

In [11]:
api = KaggleApi()
api.authenticate()

In [ ]:
archivos = []
scores = []
descripciones = []

for i in range(len(files)):

    submitted = False
    while submitted == False:
        try:
            api.competition_submit(file_name=files[i],
                                   message=descriptions[i],
                                   competition=competition)
        except Exception as e:
            print(f'Error al subir {files[i]}: {e}')
            time.sleep(1000)
        else:
            submitted = True 

            # Esperamos un poco para que kaggle no tire error
            time.sleep(30)

            submissions = api.competition_submissions(competition, page_size=1) # traigo solo la ultima
            ultima = submissions[0]
            
            archivo = ultima.fileName
            public_score = ultima.publicScore
            descripcion = ultima.description
            
            archivos.append(archivo)
            scores.append(public_score)
            descripciones.append(descripcion)
            
            print(f'{archivo} subido con exito, public score:{public_score}')

100%|██████████| 2.09M/2.09M [00:01<00:00, 1.28MB/s]


In [ ]:
# si falla y precisamos exportar resultados intermedios

# resultados = pd.DataFrame({
#     'Archivo': archivos,
#     'Public score': scores,
#     'Description':descripciones
# })

# resultados.to_excel(f'C:/Users/eaneg/OneDrive/Desktop/scores_{version}.xlsx', index=False)

# resultados = pd.read_excel(f'C:/Users/eaneg/OneDrive/Desktop/scores_{version}.xlsx')

In [ ]:
resultados = pd.DataFrame({
    'Archivo': archivos,
    'Public score': scores,
    'Description':descripciones
})

df = pd.DataFrame({
    'seed': np.repeat(semillas, len(puntos_corte)),
    'corte': np.tile(puntos_corte, len(semillas))
})

resultados = pd.concat([resultados, df], axis=1)

In [ ]:
resultados.to_excel(f'{files_path}scores_{version}.xlsx', index=False)